In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
import spacy

In [2]:
data_dir='0819_keyword_analysis.csv'
data=pd.read_csv(data_dir,encoding='utf-8')

In [4]:
data

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,create_to_view_duration,create_to_talk_duration,create_to_purchase_duration,view_to_talk_duration,view_to_purchase_duration,talk_to_purchase_duration,total_view,final_purchase_date,purchase_duration,talk_duration
0,3ef9b871794aad8e8b9a7ecff4198005,3,2023-12-11 19:45:00,에어포스 블랙 275,"에어포스275,나이키,나이키에어포스블랙",80000,21,0,0,0,...,0.0,1.0,4660.0,1.0,4.0,3.0,137,2023-12-15 01:25:00,77.666667,0
1,9d102d7cb403e706afd72802c569c4f6,3,2024-03-05 20:59:00,[260] 뉴발란스 993 그레이,"뉴발란스,993,260",300000,20,1,0,0,...,0.0,1.0,11212.0,1.0,8.0,7.0,67,2024-03-13 15:51:00,186.866667,0
2,9529223602a6babb9620fdbf3d2e9d12,3,2023-12-31 07:11:00,[265] 나이키 범고래 덩크로우 레트로,"나이키,덩크로우,범고래,덩크,조던",35000,22,1,0,0,...,0.0,0.0,6258.0,0.0,4.0,4.0,64,2024-01-04 15:29:00,104.300000,-1
3,fa317e264af77cf10675c52f84c4dbfc,3,2024-01-17 11:27:00,뉴발993 gray 2E 255size,"뉴발993,뉴발란스993,뉴발993그레이",310000,2,0,0,0,...,0.0,2.0,8111.0,2.0,6.0,4.0,5,2024-01-23 02:38:00,135.183333,1
4,4720e3f8f4b48ebf54867980f3e3431c,3,2024-04-28 20:27:00,뉴발란스 993 8d(260),NaN,160000,21,1,0,0,...,0.0,5.0,6361.0,5.0,5.0,0.0,58,2024-05-03 06:28:00,106.016667,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,db0564ef098358e074b1f8c3b212f7ad,2,2024-03-02 17:10:00,나이키 덩크로우 범고래 255 사이즈 판매합니다,"나이키,덩크로우,범고래,나이키범고래,덩크로우범고래",72000,20,1,0,0,...,0.0,2.0,2640.0,2.0,2.0,0.0,20,2024-03-04 13:10:00,44.000000,1
857,5bc19bead06de966c989b4a198535030,3,2023-12-24 23:03:00,[245 새상품] 아디다스 삼바 OG,"삼바,아디다스,삼바245",130000,2,0,0,0,...,1.0,12.0,17775.0,11.0,12.0,1.0,32,2024-01-06 07:18:00,296.250000,11
858,c55fb5c7e1120bb00043fcb61e9bb2ac,3,2023-12-08 19:26:00,아디다스 슈퍼스타 270,"아디다스슈퍼스타,슈퍼스타",55000,22,1,0,0,...,0.0,1.0,1169.0,1.0,1.0,0.0,16,2023-12-09 14:55:00,19.483333,0
859,05c53158a3cdbed45acc738d49ad113f,3,2024-02-13 14:16:00,새상품)나이키 에어포스 275,"나이키,에어포스,에어포스275,275",125000,2,0,0,0,...,0.0,0.0,1192.0,0.0,1.0,1.0,12,2024-02-14 10:08:00,19.866667,-1


In [12]:
nlp = spacy.load("ko_core_news_sm")
import ko_core_news_sm
nlp = ko_core_news_sm.load()
doc = nlp("이것은 문장입니다.")
print([(w.text, w.pos_) for w in doc])

[('이것은', 'PRON'), ('문장입니다', 'VERB'), ('.', 'PUNCT')]


In [7]:
data.columns

Index(['id', 'status', 'create_date', 'name', 'keyword', 'price', 'flag_used',
       'flag_taekpo', 'flag_exchg', 'qty', 'category_id', 'image_count',
       'modified_at', 'latest_updated_at', 'description', 'category', 'name.1',
       'is_enable', 'name_kor', 'seller_birth_year', 'seller_gender',
       'buyer_birth_year', 'buyer_gender', 'uid', 'is_active', 'created_at',
       'setl_done_date', 'purchase_confirm_date', 'total_price',
       'talk_user_count', 'first_talk_date', 'sell_count',
       'register_prod_count', 'pid', 'first_view_date', 'purchase_date',
       'create_to_view_duration', 'create_to_talk_duration',
       'create_to_purchase_duration', 'view_to_talk_duration',
       'view_to_purchase_duration', 'talk_to_purchase_duration', 'total_view',
       'final_purchase_date', 'purchase_duration', 'talk_duration'],
      dtype='object')

In [8]:
data_desc=data[['id','name','keyword','flag_taekpo','flag_used','flag_exchg','description','name_kor']]

In [15]:
import spacy
from spacy.matcher import Matcher

# spaCy 한국어 모델 로드
nlp = spacy.load("ko_core_news_sm")

def extract_information_nlp(description):
    doc = nlp(description)
    info = {}

    # 제품 상태 (상태에 관련된 단어 추출)
    status_keywords = ['좋은 상태', '거의 새 것', '거의 새것', '사용감 있음', '보관만', '상태 좋아요']
    info['제품 상태'] = '정보 없음'
    for token in doc:
        if any(keyword in token.text for keyword in status_keywords):
            info['제품 상태'] = token.text
            break

    # 거래 조건
    condition_keywords = ['직거래', '택배거래', '안전 거래']
    info['거래 조건'] = '정보 없음'
    for token in doc:
        if any(keyword in token.text for keyword in condition_keywords):
            info['거래 조건'] = token.text
            break

    # 택배비 포함 여부(배송비)
    shipping_keywords = ['배송비 포함', '배송비 별도', '택배비 포함', '택배비 별도']
    info['택배비 포함 여부'] = '정보 없음'
    for token in doc:
        if any(keyword in token.text for keyword in shipping_keywords):
            info['택배비 포함 여부'] = token.text
            break

    # 교환, 환불 조건
    refund_keywords = ['교환 가능', '교환 불가', '환불 가능', '환불 불가']
    info['교환, 환불 조건'] = '정보 없음'
    for token in doc:
        if any(keyword in token.text for keyword in refund_keywords):
            info['교환, 환불 조건'] = token.text
            break

    # 착용 횟수 (정규 표현식으로 매칭)
    wear_count_pattern = re.compile(r'(\d+회|한번|두번|세번|네번|한 번|두 번|세 번|네 번)')
    wear_count_match = wear_count_pattern.search(description)
    if wear_count_match:
        info['착용 횟수'] = wear_count_match.group(0)
    else:
        info['착용 횟수'] = '정보 없음'

    # 제품 정보 (제품명, 사이즈 등)
    info['제품 정보'] = '정보 없음'
    product_info_pattern = [{"IS_ALPHA": True}, {"IS_PUNCT": True}, {"IS_ALPHA": True}, {"IS_PUNCT": True}, {"LIKE_NUM": True}]
    matcher = Matcher(nlp.vocab)
    matcher.add("ProductInfo", [product_info_pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        info['제품 정보'] = doc[start:end].text
        break

    # 네고 가능 여부
    nego_keywords = ['네고 가능', '가격 협상 가능', '가격 조정 가능', '네고 불가', '가격 협상 불가']
    info['네고 가능 여부'] = '정보 없음'
    for token in doc:
        if any(keyword in token.text for keyword in nego_keywords):
            info['네고 가능 여부'] = token.text
            break

    # 추가 정보 (박스 등)
    info['additional_info'] = '정보 없음'
    if "박스" in description:
        info['additional_info'] = "박스 있음"

    return info

# 예시 사용
description = """
Nike Dunk Low PRM Head to Head

나이키 공홈에서 구매했습니다.
구매하고 바로 슈브이 도포한 상태입니다.
실착 10회 미만이고 사용감이 적은 상태입니다.
사이즈 270

궁금한 점 최대한 답변드리겠습니다.
"""

extracted_info = extract_information_nlp(description)
for key, value in extracted_info.items():
    print(f"{key}: {value}")


제품 상태: 정보 없음
거래 조건: 정보 없음
택배비 포함 여부: 정보 없음
교환, 환불 조건: 정보 없음
착용 횟수: 10회
제품 정보: 정보 없음
네고 가능 여부: 정보 없음
additional_info: 정보 없음


In [19]:
import spacy
from spacy import displacy
doc = nlp("She ate the pizza")
displacy.serve(doc, style="dep")

c:\Users\sunwoong\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [23]:
import spacy
import ko_core_news_sm

# 한국어 모델 로드
nlp = ko_core_news_sm.load()

# 텍스트 분석
text = "새제품으로 상태는 아주 좋습니다. 직거래와 택배 거래 모두 가능합니다."
doc = nlp(text)

# 토큰화 및 품사 태깅
for token in doc:
    print(f"단어: {token.text}, 품사: {token.pos_}, 의존 관계: {token.dep_}")

# 문장 분석
for sent in doc.sents:
    print(f"문장: {sent.text}")


단어: 새제품으로, 품사: ADV, 의존 관계: advcl
단어: 상태는, 품사: NOUN, 의존 관계: dislocated
단어: 아주, 품사: ADV, 의존 관계: advmod
단어: 좋습니다, 품사: ADJ, 의존 관계: ROOT
단어: ., 품사: PUNCT, 의존 관계: punct
단어: 직거래와, 품사: NOUN, 의존 관계: obj
단어: 택배, 품사: NOUN, 의존 관계: compound
단어: 거래, 품사: NOUN, 의존 관계: compound
단어: 모두, 품사: ADV, 의존 관계: advmod
단어: 가능합니다, 품사: VERB, 의존 관계: ROOT
단어: ., 품사: PUNCT, 의존 관계: punct
문장: 새제품으로 상태는 아주 좋습니다.
문장: 직거래와 택배 거래 모두 가능합니다.


In [35]:
import spacy
import ko_core_news_sm

# 한국어 모델 로드
nlp = ko_core_news_sm.load()

# 텍스트 분석
text = "2/3 크림에서 구입 후 1회 단시간 착용. 박스 탭 있습니다. 편의점택배(CJ대한통운) 택포 30만"
doc = nlp(text)

# 분석된 결과를 출력
for token in doc:
    print(f"단어: {token.text}, 품사: {token.pos_}, 의존 관계: {token.dep_}, 원형: {token.lemma}")


단어: 2/3, 품사: ADV, 의존 관계: amod, 원형: 13879577541206457249
단어: 크림에서, 품사: ADV, 의존 관계: advcl, 원형: 18388857083993999872
단어: 구입, 품사: VERB, 의존 관계: acl, 원형: 17733087377399425348
단어: 후, 품사: NOUN, 의존 관계: advmod, 원형: 8546234165172790064
단어: 1회, 품사: NUM, 의존 관계: compound, 원형: 2335747974944091613
단어: 단시간, 품사: ADV, 의존 관계: advcl, 원형: 9986828769377392799
단어: 착용, 품사: VERB, 의존 관계: ROOT, 원형: 13046150082109585334
단어: ., 품사: PUNCT, 의존 관계: punct, 원형: 12646065887601541794
단어: 박스, 품사: PROPN, 의존 관계: dep, 원형: 5188871516576525112
단어: 탭, 품사: NOUN, 의존 관계: dep, 원형: 17289942701269754292
단어: 있습니다, 품사: ADJ, 의존 관계: ROOT, 원형: 15203980483716606908
단어: ., 품사: PUNCT, 의존 관계: punct, 원형: 12646065887601541794
단어: 편의점택배, 품사: NOUN, 의존 관계: compound, 원형: 7945620371813582084
단어: (, 품사: PUNCT, 의존 관계: punct, 원형: 12638816674900267446
단어: CJ대한통운, 품사: NOUN, 의존 관계: appos, 원형: 13102349233207763636
단어: ), 품사: PUNCT, 의존 관계: punct, 원형: 3842344029291005339
단어: 택포, 품사: NOUN, 의존 관계: compound, 원형: 15059592955778800488
단어: 30만, 품사: NUM, 의존 관계: ROOT

In [26]:
import spacy
import ko_core_news_sm

# 한국어 모델 로드
nlp = ko_core_news_sm.load()

def extract_info_nlp(text):
    # 초기화된 정보 사전
    info = {
        '제품 상태': None,
        '거래 조건': None,
        '구매처': None,
        '사이즈': None,
        '택배비 포함 여부(배송비)': None,
        '교환, 환불 조건': None,
        '착용 횟수': None,
        '제품 정보': None,
        '네고 가능 여부': None,
        'additional_info': None,
    }
    
    # 텍스트를 분석
    doc = nlp(text)
    
    # 각 문장을 순회하면서 특정 키워드에 대해 구체적인 정보를 추출
    for sent in doc.sents:
        # 구매처 정보 추출
        if "구입" in sent.text or "구매" in sent.text:
            info['구매처'] = sent.text.strip()
        
        # 착용 횟수 추출
        if "착용" in sent.text:
            info['착용 횟수'] = sent.text.strip()
        
        # 제품 상태 추출
        if "박스" in sent.text or "상태" in sent.text:
            info['제품 상태'] = sent.text.strip()
        
        # 택배비 포함 여부 추출
        if "택포" in sent.text or "택배비 포함" in sent.text:
            info['택배비 포함 여부(배송비)'] = sent.text.strip()
        
        # 거래 조건 추출
        if "택배" in sent.text or "직거래" in sent.text:
            info['거래 조건'] = sent.text.strip()

    return info

# 예제 문장
text = "2/3 크림에서 구입 후 1회 단시간 착용. 박스 탭 있습니다. 편의점택배(CJ대한통운) 택포 30만"
result = extract_info_nlp(text)

# 추출된 정보 출력
for key, value in result.items():
    if value:
        print(f"{key}: {value}")


제품 상태: 박스 탭 있습니다.
거래 조건: 편의점택배(CJ대한통운) 택포 30만
구매처: 2/3 크림에서 구입 후 1회 단시간 착용.
택배비 포함 여부(배송비): 편의점택배(CJ대한통운) 택포 30만
착용 횟수: 2/3 크림에서 구입 후 1회 단시간 착용.


In [29]:
import spacy
from spacy import displacy
doc = nlp("21년도 크림에서 구매했습니다. 사용감 있습니다. 편하게 운동용으로 신으실 분 가저가세요 :)")
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [33]:
import spacy
import ko_core_news_sm

# 한국어 모델 로드
nlp = ko_core_news_sm.load()

def extract_info_from_structure(text):
    info = {
        '구매처': None,
        '사용감': None,
    }
    
    # 텍스트를 분석
    doc = nlp(text)
    
    for token in doc:
        # 구매처 추출: '구매' 동사와 수식어로 연결된 구매처 정보 추출
        if token.lemma_ == '구매하다' and token.pos_ == 'VERB':
            for child in token.children:
                if '에서' in child.text:
                    info['구매처'] = child.text.strip()
        
        # 사용감 추출: '있습니다' 형용사와 주어로 연결된 사용감 정보 추출
        if token.lemma_ == '있다' and token.pos_ == 'ADJ':
            for child in token.children:
                if child.dep_ == 'nsubj' and '사용감' in child.text:
                    info['사용감'] = '있음'
    
    return info

# 예제 문장
text_1 = "크림에서 구매했습니다."
text_2 = "사용감 있습니다."

result_1 = extract_info_from_structure(text_1)
result_2 = extract_info_from_structure(text_2)

# 추출된 정보 출력
print(f"문장 1 결과: 구매처 - {result_1['구매처']}")
print(f"문장 2 결과: 사용감 - {result_2['사용감']}")


문장 1 결과: 구매처 - 크림에서
문장 2 결과: 사용감 - 있음


In [37]:
import spacy
import ko_core_news_sm

# 한국어 모델 로드
nlp = ko_core_news_sm.load()

# 예제 문장
text = "2/3 크림에서 구입 후 1회 단시간 착용. 박스 탭 있다. 편의점택배(CJ대한통운) 택포 30만"

# 텍스트 분석
doc = nlp(text)

# 각 단어의 lemma 확인
lemmas = [(token.text, token.lemma_) for token in doc]
for word, lemma in lemmas:
    print(f"단어: {word}, lemma: {lemma}")


단어: 2/3, lemma: 2/3
단어: 크림에서, lemma: 크림+에서
단어: 구입, lemma: 구입
단어: 후, lemma: 후
단어: 1회, lemma: 1+회
단어: 단시간, lemma: 단시+간
단어: 착용, lemma: 착용
단어: ., lemma: .
단어: 박스, lemma: 박스
단어: 탭, lemma: 탭
단어: 있다, lemma: 있+다
단어: ., lemma: .
단어: 편의점택배, lemma: 편의점+택배
단어: (, lemma: (
단어: CJ대한통운, lemma: CJ대한통운
단어: ), lemma: )
단어: 택포, lemma: 택포
단어: 30만, lemma: 30+만


In [40]:
import spacy
import ko_core_news_lg

# 한국어 대형 모델 로드
nlp = ko_core_news_lg.load()

# 예제 문장
text = "2/3 크림에서 구입 후 1회 단시간 착용. 박스 탭 있습니다. 편의점택배(CJ대한통운) 택포 30만"

# 텍스트 분석
doc = nlp(text)

# 각 단어의 lemma(원형) 확인
lemmas = [(token.text, token.lemma_) for token in doc]

# 결과 출력
for word, lemma in lemmas:
    print(f"단어: {word}, 원형: {lemma}")


단어: 2/3, 원형: 2/3
단어: 크림에서, 원형: 크림+에서
단어: 구입, 원형: 구입
단어: 후, 원형: 후
단어: 1회, 원형: 1+회
단어: 단시간, 원형: 단시간
단어: 착용, 원형: 착용
단어: ., 원형: .
단어: 박스, 원형: 박스
단어: 탭, 원형: 탭
단어: 있습니다, 원형: 있+습니다
단어: ., 원형: .
단어: 편의점택배, 원형: 편의점택배
단어: (, 원형: (
단어: CJ대한통운, 원형: CJ대한통운
단어: ), 원형: )
단어: 택포, 원형: 택포
단어: 30만, 원형: 30+만
